In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("../data/data.csv")

In [20]:
df.dtypes

TransactionId            object
BatchId                  object
AccountId                object
SubscriptionId           object
CustomerId               object
CurrencyCode             object
CountryCode               int64
ProviderId               object
ProductId                object
ProductCategory          object
ChannelId                object
Amount                  float64
Value                     int64
TransactionStartTime     object
PricingStrategy           int64
FraudResult               int64
dtype: object

In [21]:
# Grouping by CustomerId to create aggregate features
aggregated_features = df.groupby('CustomerId').agg({
    'Amount': ['sum', 'mean', 'max', 'std'],  # Spending patterns
    'TransactionStartTime': 'count'  # Number of transactions
})

# Rename columns for clarity
aggregated_features.columns = ['_'.join(col).strip() for col in aggregated_features.columns]
aggregated_features.reset_index(inplace=True)

aggregated_features

,CustomerId,Amount_sum,Amount_mean,Amount_max,Amount_std,TransactionStartTime_count
0,CustomerId_1,-10000.0,-10000.000000,-10000.0,NaN,1
1,CustomerId_10,-10000.0,-10000.000000,-10000.0,NaN,1
2,CustomerId_1001,20000.0,4000.000000,10000.0,6558.963333,5
3,CustomerId_1002,4225.0,384.090909,1500.0,560.498966,11
4,CustomerId_1003,20000.0,3333.333333,10000.0,6030.478146,6
...,...,...,...,...,...,...
3737,CustomerId_992,20000.0,3333.333333,10000.0,6088.240030,6
3738,CustomerId_993,20000.0,4000.000000,10000.0,6745.368782,5
3739,CustomerId_994,543873.0,5384.881188,90000.0,14800.656784,101
3740,CustomerId_996,139000.0,8176.470588,10000.0,4433.329648,17


In [22]:
import sys
sys.path.append('../scripts')

from feature_extraction import extract_transaction_features
# Extract features
df = extract_transaction_features(df, 'TransactionStartTime')
df

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Transaction Hour,Transaction Day,Transaction Month,Transaction Year
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,3,15,11,2018
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,3,15,11,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95657,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-1000.0,1000,2019-02-13 09:54:09+00:00,2,0,9,13,2,2019
95658,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2019-02-13 09:54:25+00:00,2,0,9,13,2,2019
95659,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2019-02-13 09:54:35+00:00,2,0,9,13,2,2019
95660,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,256,ProviderId_6,ProductId_19,tv,ChannelId_3,3000.0,3000,2019-02-13 10:01:10+00:00,2,0,10,13,2,2019


In [23]:
df.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
Transaction Hour        0
Transaction Day         0
Transaction Month       0
Transaction Year        0
dtype: int64

In [51]:
# Convert TransactionStartTime to datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

In [54]:
# One-Hot Encoding for ProductCategory and ChannelId
df_encoded = pd.get_dummies(df, columns=['ProductCategory', 'ChannelId'], drop_first=True)

In [55]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding for CustomerId and ProviderId
label_encoder = LabelEncoder()
df_encoded['CustomerId_Encoded'] = label_encoder.fit_transform(df_encoded['CustomerId'])
df_encoded['ProviderId_Encoded'] = label_encoder.fit_transform(df_encoded['ProviderId'])

In [60]:
from sklearn.preprocessing import MinMaxScaler

# Normalization of Amount
scaler = MinMaxScaler()
df_encoded['Amount_Normalized'] = scaler.fit_transform(df_encoded[['Amount']])

In [62]:
from sklearn.preprocessing import StandardScaler

# Standardization of Amount
scaler = StandardScaler()
df_encoded['Amount_Standardized'] = scaler.fit_transform(df_encoded[['Amount']])

In [65]:
import numpy as np

# Create RFMS score based on existing data
df_encoded['RFMS'] = (df_encoded['Amount_Normalized'] * 0.5) + (df_encoded['Transaction Hour'] * 0.25) + (df_encoded['Transaction Day'] * 0.25)

# Define threshold for classification
threshold = df_encoded['RFMS'].median()
df_encoded['Label'] = np.where(df_encoded['RFMS'] > threshold, 'Good', 'Bad')
print(df_encoded[['CustomerId', 'RFMS', 'Label']])

            CustomerId      RFMS Label
0      CustomerId_4406  4.296002   Bad
1      CustomerId_4406  4.295955   Bad
2      CustomerId_4683  4.295979   Bad
3       CustomerId_988  4.546875   Bad
4       CustomerId_988  4.545926   Bad
...                ...       ...   ...
95657  CustomerId_3078  5.545910   Bad
95658  CustomerId_3874  5.546002   Bad
95659  CustomerId_3874  5.545955   Bad
95660  CustomerId_1709  5.796094   Bad
95661  CustomerId_1709  5.795953   Bad

[95662 rows x 3 columns]


In [80]:
df_encoded['Label'].value_counts()

Label
0    47842
1    47820
Name: count, dtype: int64

In [81]:
# Calculate WoE
def calculate_woe(df, target, feature):
    # Create a DataFrame for WoE calculation
    woe_df = df.groupby(feature)[target].agg(['count', 'sum']).reset_index()
    woe_df.columns = [feature, 'Total', 'Good']
    woe_df['Bad'] = woe_df['Total'] - woe_df['Good']
    
    # Calculate total good and bad
    total_good = df[target].sum()
    total_bad = df[target].count() - total_good
    
    # Calculate WoE
    woe_df['WoE'] = np.log((woe_df['Good'] / total_good) / (woe_df['Bad'] / total_bad))
    
    return woe_df[[feature, 'WoE']]

# Apply WoE calculation
woe_result = calculate_woe(df_encoded, 'Label', 'ProductCategory')
print(woe_result)

KeyError: 'ProductCategory'

In [78]:
# Merge WoE values into the original DataFrame
df_with_woe = df_encoded.merge(woe_result, on='ProductCategory', how='left')
print(df_with_woe[['ProductCategory', 'WoE', 'Label_Binary']])

KeyError: 'ProductCategory'